# Multivariate Regression to predict fish weight
https://www.kaggle.com/datasets/aungpyaeap/fish-market


### About the Data Set

### About Multivariate Regression

Multivariate (or single variable) regression, allows us to predict our predictor variable based on the values of one or more independent variables. 

$ \hat{y} = b_1 * x_1 + b_2 * x_2... b_k * x_k $

After generating the coefficients of the model to predict $ \hat{y} $ , we find $ R $  which measures the correlation between the predictor variable and the indepedent variables. Then, we can use adjusted $ R^2 $ to see how much the predictor variable can be explained the the independent variables. We must use adjusted $ R^2 $ because $ R^2 $ overestimates when multiple independent variables are used. Next, we must interpret the standard estimation error, which tells us, on average, how much the model overestimates the predictor variable. 

Next, we must statistically verify our model. We can use an F-test to test the null hypothesis, which is that all coeffients are 0, i.e. they have no affect on the predictor variable. 

Lastly, we must look at the standardized coeffieints' beta values to see which independent variables have the most influence on our predictor variable. We also need to check the p-value for statistical significance. 

https://www.youtube.com/watch?v=sDrAoR17pNM

#### Assumptions
1. Must be linear relationship between dependent and independent variables.
2. Error must be normally distributed (kolmorogrov smirnov test or graph as histogram)
3. No multicollinearity (if there is it means coefficients are unstbale)
4. No heteroskedasticity (variance of the residuals must be constant across predicted values)

### Exploratory Data Analysis

In [53]:
# Import required libraries


# Import Dataset 


Now, we will check that are dataset meets all the assumptions for multilinear regression. 

1. Must be linear relationship between dependent and independent variables.
2. Error must be normally distributed (kolmorogrov smirnov test or graph as histogram)
3. No multicollinearity (if there is it means coefficients are unstbale)
4. No heteroskedasticity (variance of the residuals must be constant across predicted values)

### Multivariable Regression

https://www.educative.io/answers/how-to-implement-multivariable-regression-in-python


R^2 and p-value  ANOVA

### Regression Error - epsilon

### Multivariable vs. Singular Regression

### Conclusion

the relationships between variables in one's background are so complex that multivriable regression is a better predictor 